### Dataset -> Prompted Training -> SQL Generation -> Excution Evaluation

## Stage 0 — Environment & constraints (do this once)

Goal: Make iteration fast and failure cheap.
	•	Single GPU (Colab / local)
	•	fp16 or bf16
	•	LoRA adapters only
	•	Small subset of WikiSQL for now (e.g. 5–10%)

Key principle:
Training is an experiment, not a ceremony.

In [1]:
!pip install torch torchvision torchaudio transformers accelerate datasets peft bitsandbytes tqdm sqlalchemy

In [2]:
import torch
import random
import numpy as np

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)


## Stage 1 — Dataset preparation (WikiSQL, but disciplined)

What you load

From WikiSQL:
	•	question
	•	table schema
	•	sql (ground truth)
	•	db_id

Ignore joins for now — WikiSQL is single-table by design. This is a feature, not a limitation.

In [5]:
import os
BASE_DIR = "/content/text2sql"

DIRS = [
    "data/wikisql",
    "outputs/logs",
    "adapters"
]

for d in DIRS:
    os.makedirs(os.path.join(BASE_DIR, d), exist_ok=True)
BASE_DIR

'/content/text2sql'

In [6]:
import torch
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

Tesla T4


In [7]:
from datasets import load_dataset

dataset = load_dataset("htriedman/wikisql")
dataset


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 56355
    })
    validation: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 8421
    })
    test: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 15878
    })
})

In [ ]:
train_raw = dataset["train"].shuffle(seed=SEED).select(range(1000))
val_raw   = dataset["validation"].shuffle(seed=SEED).select(range(200))
test_raw  = dataset["test"].shuffle(seed=SEED).select(range(200))

In [9]:
def format_schema(table):
  table_name = table["name"]
  columns = table["header"]
  return f"Table : {table_name} \n Columns : {', '.join(columns)} \n"

### Canonical prompt bulder

In [10]:
def build_prompt(instruction, schema_string):
  # The schema is now a pre-formatted string in 'input', so format_schema is not needed here
  return (
        "You are an expert SQL generator.\n\n"
        "### Database Schema:\n"
        f"{schema_string}\n\n"
        "### Question:\n"
        f"{instruction}\n\n"
        "### SQL:\n"
  )


### SQL target excutions

In [11]:
def extract_sql(example):
    # The SQL is now in the 'output' field
    return example["output"]


### preproccessing function

In [12]:
def preprocess(example):
    # Map 'instruction' to question and 'input' to schema_string
    prompt = build_prompt(example["instruction"], example["input"])
    sql = extract_sql(example)
    return {
        "text": prompt + sql
    }


### apply mapping

In [13]:
train_data = train_raw.map(
    preprocess,
    remove_columns=train_raw.column_names
)

val_data = val_raw.map(
    preprocess,
    remove_columns=val_raw.column_names
)

In [14]:
# check
print(train_data[0]["text"])

You are an expert SQL generator.

### Database Schema:
Which sum of week that had an attendance larger than 55,767 on September 28, 1986?

### Question:
Translate the following into a SQL query

### SQL:
SELECT SUM Week FROM table WHERE Attendance > 55,767 AND Date = september 28, 1986


## Stage 2 — Model setup (Mistral 7B, correctly constrained)

Model choice
	•	mistralai/Mistral-7B-v0.1 (or instruct if available)

You are not instruction-tuning broadly.
You are task-conditioning narrowly.

⸻

LoRA configuration (conceptual)
	•	Target: attention + FFN layers
	•	Rank: small (8–16)
	•	Trainable params: <1%
	•	Base weights frozen

This keeps:
	•	Training under 1 hour
	•	Behavior stable
	•	Overfitting controlled

⸻

Training scope (for now)
	•	5–10% of WikiSQL
	•	1–2 epochs max
	•	Small batch size
	•	No hyperparameter obsession yet

Right now you’re asking:

“Does this architecture even learn?”

Not:

“Is this SOTA?”

### Load Tokenizer

In [15]:
from transformers import AutoTokenizer

MODEL_NAME = "mistralai/Mistral-7B-v0.1"

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    use_fast=True
)

# Important: Mistral has no pad token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

tokenizer.padding_side = "right"

##  Load model in 4-bit

In [16]:
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

MODEL_NAME = "mistralai/Mistral-7B-v0.1"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
)

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/291 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

### prepare model for LoRA training

In [17]:
from peft import prepare_model_for_kbit_training

model = prepare_model_for_kbit_training(model)

## attach loRA adapters

In [18]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=8,                      # low rank → fast
    lora_alpha=16,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)

In [20]:
# verify trainable parameters
model.print_trainable_parameters()

trainable params: 20,971,520 || all params: 7,262,703,616 || trainable%: 0.2888


## Stage 3 — Training loop (minimalist, intentional)

What happens conceptually:
	•	Model reads schema + question
	•	Learns to continue with SQL
	•	Loss only applies to SQL tokens

No magic. No reinforcement tricks.

⸻

Guardrails
	•	Stop training early if loss plateaus
	•	Save LoRA adapter only
	•	Log a few decoded samples per epoch

If the model starts inventing columns → your prompt is broken, not the model.

### Tokenizatin function

In [ ]:
MAX_LENGTH = 256
def tokenize_fn(example):
  return tokenizer(
      example["text"],
      truncation = True,
      max_length = MAX_LENGTH,
      padding = "max_length"
  )

In [22]:
# apply tokenization

train_tokenized = train_data.map(
    tokenize_fn,
    batched = True,
    remove_columns=["text"]
)

val_tokenized = val_data.map(
    tokenize_fn,
    batched = True,
    remove_columns=["text"]
)


Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

### Data collator

In [23]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer = tokenizer,
    mlm = False
)

### Training configuration for short runs, stability and fast feedbacks

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="/content/text2sql/outputs",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=4,  # effective batch size = 4
    learning_rate=2e-4,
    num_train_epochs=1,             # FAST TEST RUN
    max_steps=200,                  # cap total steps for speed
    fp16=True,
    logging_steps=25,
    eval_strategy="steps", # Changed from evaluation_strategy to eval_strategy
    eval_steps=100,
    save_steps=100,
    save_total_limit=1,
    report_to="none",
    remove_unused_columns=False,
 )

## trainer setup

In [28]:
from transformers import Trainer

trainer = Trainer (
    model = model,
    args = training_args,
    train_dataset = train_tokenized,
    eval_dataset = val_tokenized,
    data_collator = data_collator,
)

In [ ]:
## train
trainer.train()

/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss,Validation Loss


## Stage 4 — Inference pipeline (this is NOT training)

For each test example:
	1.	Load schema + question
	2.	Format prompt exactly like training
	3.	Generate SQL with:
	•	Low temperature (≈ 0.1–0.2)
	•	Limited max tokens
	4.	Stop generation at newline or EOS

No post-processing beyond trimming.

This preserves scientific honesty.

## Stage 5 — Execution accuracy evaluation (the only metric that matters)

Why execution accuracy

Exact string match is a lie detector for humans, not machines.

Execution accuracy asks:

“Did this SQL answer the question?”

That’s what users care about.

## Stage 6 — Analysis (this is where intelligence shows)

After the first run, you analyze:
	•	Does it fail on:
	•	Aggregations?
	•	WHERE clauses?
	•	Column selection?
	•	Are errors systematic or random?
	•	Does it hallucinate schema elements?

This tells you:
	•	Whether more data helps
	•	Or schema representation needs improvement
	•	Or prompt structure needs revision

# Stage 7 — Scaling later (explicitly postponed)

Only after the pipeline works:
	•	Train on full WikiSQL
	•	Increase epochs
	•	Possibly curriculum learning
	•	Optionally move toward Spider-easy